## Train MM / explore with random sampling for HP tuning


In [ ]:
import matplotlib

matplotlib.use("Agg")

import logging

logger = logging.getLogger()
logger.disabled = True

import os
from agent import DQNAgent
from tqdm.auto import tqdm
import random
import itertools

# Number of combinations you want
num_combinations = 5  # Change this to however many combinations you need

# default
room_size = "xl-different-prob"
capacity_max = 6
batch_size = 32
terminates_at = 99
num_iterations = (terminates_at + 1) * 100
validation_starts_at = num_iterations // 2

prob_type = (
    "non-equal-object-probs" if "different-prob" in room_size else "equal-object-probs"
)
root_path = (
    f"./training-results/{prob_type}/dqn/room_size={room_size}/capacity={capacity_max}/"
)

# random
test_seed_ = [i for i in range(100)]
target_update_interval_ = [10]
min_epsilon_ = [0.1]
gamma_mm_ = [0.99]
gamma_explore_ = [0.9]
semantic_decay_factor_ = [0.70]

# Weights for agent_capacity_ elements
agent_capacity_weights = [0, 0, 1]
agent_capacity_ = [
    {"episodic": capacity_max // 2, "semantic": capacity_max // 2, "short": 1},
    {"episodic": capacity_max, "semantic": 0, "short": 1},
    {"episodic": 0, "semantic": capacity_max, "short": 1},
]
replay_buffer_size_ = [
    # num_iterations,
    num_iterations
    // 2,
]
warm_start_ = [
    # num_iterations // 2,
    num_iterations // 4,
    # num_iterations // 10,
]


# Generate all combinations
params_all = list(
    itertools.product(
        test_seed_,
        target_update_interval_,
        min_epsilon_,
        gamma_mm_,
        gamma_explore_,
        semantic_decay_factor_,
        replay_buffer_size_,
        warm_start_,
    )
)

# Random combinations with weighted agent_capacity_
random_combinations = random.sample(params_all, num_combinations)
random_agent_capacity = random.choices(
    agent_capacity_, weights=agent_capacity_weights, k=num_combinations
)


for i, params in tqdm(enumerate(random_combinations)):
    (
        test_seed,
        target_update_interval,
        min_epsilon,
        gamma_mm,
        gamma_explore,
        semantic_decay_factor,
        replay_buffer_size,
        warm_start,
    ) = params

    capacity = random_agent_capacity[i]

    params_dict = {
        "env_str": "room_env:RoomEnv-v2",
        "num_iterations": num_iterations,
        "replay_buffer_size": replay_buffer_size,
        "validation_starts_at": validation_starts_at,
        "warm_start": warm_start,
        "batch_size": batch_size,
        "target_update_interval": target_update_interval,
        "epsilon_decay_until": num_iterations,
        "max_epsilon": 1.0,
        "min_epsilon": min_epsilon,
        "gamma": {"mm": gamma_mm, "explore": gamma_explore},
        "capacity": capacity,
        "pretrain_semantic": False,
        "semantic_decay_factor": semantic_decay_factor,
        "lstm_params": {
            "num_layers": 2,
            "embedding_dim": 64,
            "hidden_size": 64,
            "bidirectional": False,
            "max_timesteps": terminates_at + 1,
            "max_strength": terminates_at + 1,
            "relu_for_attention": True,
        },
        "mlp_params": {
            "hidden_size": 64,
            "num_hidden_layers": 1,
            "dueling_dqn": True,
        },
        "num_samples_for_results": {"val": 5, "test": 10},
        "validation_interval": 5,
        "plotting_interval": 50,
        "train_seed": test_seed + 5,
        "test_seed": test_seed,
        "device": "cpu",
        "qa_function": "episodic_semantic",
        "explore_policy_heuristic": "avoid_walls",
        "env_config": {
            "question_prob": 1.0,
            "terminates_at": terminates_at,
            "randomize_observations": "objects",
            "room_size": room_size,
            "rewards": {"correct": 1, "wrong": 0, "partial": 0},
            "make_everything_static": False,
            "num_total_questions": 1000,
            "question_interval": 1,
            "include_walls_in_observations": True,
        },
        "ddqn": True,
        "default_root_dir": root_path,
    }

    agent = DQNAgent(**params_dict)
    agent.train()

## Run fixed combinations

In [ ]:
import matplotlib

matplotlib.use("Agg")

import logging

logger = logging.getLogger()
logger.disabled = True

import os
from agent import DQNAgent
from tqdm.auto import tqdm
import random
import itertools

room_size = "xl-different-prob"
terminates_at = 99
num_iterations = (terminates_at + 1) * 100
replay_buffer_size = num_iterations
validation_starts_at = 0
warm_start = num_iterations // 10
batch_size = 32
target_update_interval = 10
gamma_mm = 0.99
gamma_explore = 0.9


for capacity_max in [12]:
    prob_type = (
        "non-equal-object-probs"
        if "different-prob" in room_size
        else "equal-object-probs"
    )
    root_path = (
        f"./training-results/{prob_type}/dqn/"
        f"room_size={room_size}/capacity={capacity_max}/"
    )
    for test_seed in [0, 1, 2, 3, 4]:

        for agent_type in ["hybrid", "semantic", "episodic"]:

            if agent_type == "hybrid":
                capacity = {
                    "episodic": capacity_max // 2,
                    "semantic": capacity_max // 2,
                    "short": 1,
                }
                semantic_decay_factor_ = [0.70, 0.90, 0.99]
                pretrain_semantic_ = [False]
            elif agent_type == "episodic":
                capacity = {"episodic": capacity_max, "semantic": 0, "short": 1}
                semantic_decay_factor_ = [1.0]
                pretrain_semantic_ = [False]
            elif agent_type == "semantic":
                capacity = {"episodic": 0, "semantic": capacity_max, "short": 1}
                semantic_decay_factor_ = [0.70, 0.90, 0.99]
                pretrain_semantic_ = [False]
            else:
                raise ValueError(f"Unknown agent_type: {agent_type}")

            for pretrain_semantic in pretrain_semantic_:
                for semantic_decay_factor in semantic_decay_factor_:
                    params_dict = {
                        "env_str": "room_env:RoomEnv-v2",
                        "num_iterations": num_iterations,
                        "replay_buffer_size": replay_buffer_size,
                        "validation_starts_at": validation_starts_at,
                        "warm_start": warm_start,
                        "batch_size": batch_size,
                        "target_update_interval": target_update_interval,
                        "epsilon_decay_until": num_iterations,
                        "max_epsilon": 1.0,
                        "min_epsilon": 0.1,
                        "gamma": {"mm": gamma_mm, "explore": gamma_explore},
                        "capacity": capacity,
                        "pretrain_semantic": pretrain_semantic,
                        "semantic_decay_factor": semantic_decay_factor,
                        "lstm_params": {
                            "num_layers": 2,
                            "embedding_dim": 64,
                            "hidden_size": 64,
                            "bidirectional": False,
                            "max_timesteps": terminates_at + 1,
                            "max_strength": terminates_at + 1,
                            "relu_for_attention": True,
                            "use_one_hot": True,
                        },
                        "mlp_params": {
                            "hidden_size": 64,
                            "num_hidden_layers": 1,
                            "dueling_dqn": True,
                        },
                        "num_samples_for_results": {"val": 10, "test": 10},
                        "validation_interval": 1,
                        "plotting_interval": 50,
                        "train_seed": test_seed + 5,
                        "test_seed": test_seed,
                        "device": "cpu",
                        "qa_function": "episodic_semantic",
                        "explore_policy_heuristic": "avoid_walls",
                        "env_config": {
                            "question_prob": 1.0,
                            "terminates_at": terminates_at,
                            "randomize_observations": "objects",
                            "room_size": room_size,
                            "rewards": {"correct": 1, "wrong": 0, "partial": 0},
                            "make_everything_static": False,
                            "num_total_questions": 1000,
                            "question_interval": 1,
                            "include_walls_in_observations": True,
                        },
                        "ddqn": True,
                        "default_root_dir": root_path,
                    }

                    agent = DQNAgent(**params_dict)
                    agent.train()